<a href="https://colab.research.google.com/github/jiwoong2/deeplearning/blob/main/%EC%8B%AC%EC%B8%B5%EC%8B%A0%EA%B2%BD%EB%A7%9D_%EA%B5%AC%ED%98%84.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
from collections import OrderedDict

# Layers

In [ ]:
# 4차원 텐서를 염두해둔 코드이기 떄문에 다음 학기때완전히 이해가능
class Affine:
    def __init__(self, W, b):
        self.W = W # 가중치 행렬
        self.b = b # bias 벡터

        # 입력된 data 초기화
        self.x = None
        self.original_x_shape = None
        # 가중치와 bias 미분
        self.dW = None
        self.db = None

    def forward(self, x):
        # 텐서 대응
        self.original_x_shape = x.shape
        x = x.reshape(x.shape[0], -1) # 4차원 텐서를 행렬로 변환. -1을 입력하면 reshape 가능한 숫자를 알아서 대입한다.
        self.x = x

        out = np.dot(self.x, self.W) + self.b # affine 변환. bias의 경우 배치처리를 할경우 행렬뎃섬에서 shape이 맞지 않는데 numpy에서 자동으로 벡터를 복사해 각 열에 더하게 된다.

        return out

    def backward(self, dout): # dout:흘러들어온 미분
        dx = np.dot(dout, self.W.T)
        self.dW = np.dot(self.x.T, dout)
        self.db = np.sum(dout, axis=0) # 행렬의 경우 axis 0: 행, 1: 열. 텐서의 경우 axis 0: 각 행렬, 1: 각 행렬의 행, 2: 각 행렬의 열

        dx =dx.reshape(*self.original_x_shape) # 입력데이터

        return dx

In [ ]:
class Sigmoid:
    def __init__(self):
        self.out = None

    def forward(self, x):
        out = 1 / (1 + np.exp(-x))
        self.out = out
        return out

    def backward(self, dout):
        dx = dout * (1.0 - self.out) * self.out
        return dx

In [ ]:
class Relu:
    def __init__(self):
        self.mask = None

    def forward(self, x):
        self.mask = (x <= 0) # self.mask는 0보다 크면 false 0보다 작으면 true인 리스트를 반환한다.
        out = x.copy()
        out[self.mask] = 0 # 설명1

        return out

    def backward(self, dout):
        dout[self.mask] = 0 # 위의 설명1을 참조. 주의할점은 흘러들어온 미분값의 부호에 의해 1이나 0을 곱하는것이 정해지는게 아니라 위의 x값을 기준으로 gradient를 살리거나 죽이게 된다.
        dx = dout

        return dx

In [ ]:
def cross_entropy_error(y, t):
    if y.ndim == 1: # y가 벡터인 경우, 배치처리를 안 한 경우
        t = t.reshape(1, t.size) # 10차원 벡터를 행렬로 변환.(개념상.)
        y = y.reshape(1, y.size)

    if t.size == y.size: # 라벨이 원-핫 인코딩이 돼어 있다면, 그러니까 입력 값이 10차원 벡터라면, size함수는 메트릭스의 원소갯수를 반환한다.
        t = t.argmax(axis=1) # 원-핫 인코딩 이전으로 되돌리기 위해 작성.

    batch_size = y.shape[0]
    return -np.sum(np.log(y[np.arange(batch_size), t])) / batch_size # 설명1, 확률벡터로 이루어진 행렬에서 알맞은 인덱스를 얻기위한 과정.

def softmax(x):
    if x.ndim == 2:   # x가 행렬일 경우, 즉 배치처리를 해서 각 행이 확률벡터인 메트릭스를 입력받은 경우.
        x = x.T # 각 행이 확률벡터인 초기 메트릭스에서 각 열이 확률벡터인 메트릭스로 전치 시킨다.
        x = x - np.max(x, axis = 0) # np.max(x, axis = 0)는 각 열에 대한 최댓값을 저장한 리스트(벡터)를 반환 한다. 그리고 x에 대한 - 연산은 각 열의 확률벡터에 각 열의 최댓값을 빼주는 연산이다.(밑의 오버플로우 방지를 위한 최댓값 빼기와 같음.)
        y = np.exp(x) / np.sum(np.exp(x), axis = 0) # 위와 같은 행렬과 벡터의 연산 원리.( 밑의 예제 참고.)
        return y.T

    x = x - np.max(x) # 오버플로를 방지하기위해 작성. 노트참고. , vector에 scalar를 빼면 numpy에서 알아서 원소별로 연산을 한다.
    return np.exp(x) / np.sum(np.exp(x))  # normalize

class SoftmaxWithLoss:
    def __init__(self):
        self.loss = None #손실함수
        self.y = None # softmax의 출력(확률 벡터)
        self.t = None # 정답 레이블(one-hot encoding 형태)

    def forward(self, x,  t):
        self.t = t
        self.y = softmax(x)
        self.loss = cross_entropy_error(self.y, self.t)

        return self.loss

    def backward(self, dout = 1): # 역전파의 시작은 1 이다.
        batch_size = self.t.shape[0]

        if self.t.size == self.y.size: # 정답 레이블이 원-핫 인코딩이 돼어 있는 경우.
            dx = (self.y - self.t) / batch_size # 손실함수의 정의 값은 각각의 손실함수값의 평균이므로 배치사이즈로 나누어 준다.

        else: # 원-핫 이코딩이 돼어 있지 않은 경우.
            dx = self.y.copy()
            dx[np.arange(batch_size), self.t] -= 1 # 도출된 확률벡터는 배치처리로 묶여있다는 것을 생각해보면 위와 같은 결과를 도출함.
            dx = dx / batch_size

            return dx

In [ ]:
class BatchNormalization:

    def __init__(self, gamma, beta, momentum = 0.9, running_mean = None, running_var = None): # gamma와 beta는 머신이 학습해 최적값을 찾지만 초기값을 설정해 줘야 한다.
                                                                                              # running_mean = None, running_var = None : 학습중엔 각 배치행렬(data 묶음)의 열(feature)을 사용해
                                                                                              # 평균과, 분산을 구해 사용하지만 학습이 끝난후엔 학습하는 동안의 전체 훈련용 data의 평균과 분산(비슷한 값?)
                                                                                              # 을 사용해 normalizetion 한다.
        self.gamma = gamma
        self.beta = beta
        self.momentum = momentum
        self.input_shape = None # 합성곱 계층은 4차원, 완전연결 계층은 2차원

        # test용 평균, 분산
        self.running_mean = running_mean
        self.running_var = running_var

        # backward 시에 사용할 중간 데이터
        self.batch_size = None
        self.xc = None
        self.std = None
        self.dgamma = None
        self.dbeta = None

    def forward(self, x, train_flg = True): # x는 입력 data, train_flag : 위의 주석처럼 훈련시와 test시의 순전파방식이 다르기 때문에 훈련시 train_flg를 True로 지정해 이를 구분한다.
        self.input_shape = x.shape
        if x.ndim != 2: # 행렬이 아니면. 지금까지는 data를 flatten한후 배치처리 했기때문에 인풋이 행렬이었지만, cnn등에서는 data를 faltten시키지 않기때문에 행렬이 아니라 텐서가 입력된다.
                        # N은 배치사이즈, C는 채널, H는 세로해상도, W는 가로해상도
            N, C, H, W = x.shape
            x = x.reshape(N, -1)

        out = self.__forward(x, train_flg)

        return out.reshape(*self.input_shape) # shape을 원래대로 복원한다. *는 ()를 없에준다.

    def __forward(self, x, train_flg):
        if self.running_mean is None:
            N, D = x.shape
            self.running_mean = np.zeros(D)
            self.running_var = np.zeros(D)

        if train_flg: # 훈련시.
            mu = x.mean(axis = 0) # 열의 평균
            xc = x - mu # 각 열의 평균이 0이 됨.
            var = np.mean(xc**2, axis = 0) # 분산
            std = np.sqrt(var + 10e-7) # 표준편차
            xn = xc / std # 각 열의 표준편차가 1이됨.

            self.batch_size = x.shape[0]
            self.xc = xc
            self.xn = xn
            self.std = std
            self.running_mean = self.momentum * self.running_mean + (1 - self.momentum) * mu # RMSProp에서 사용한 기법처럼 평균벡터를 계속해서 내분하는 과정. 결과적으로 계속해서 바뀌는 배치묶음의 평균 추세를 따라간다.
            self.running_var = self.momentum * self.running_var + (1 - self.momentum) * var

        else: # train_flg가 false일 경우 즉, test시에는 running_mean와 running_var을 사용한다.
            xc = x - self.running_mean
            xn = xc / ((np.sqrt(self.running_var + 10e-7)))

        out = self.gamma * xn + self.beta # gamma를 곱해 표준편차를 조정하고 beta를 더해 평균을 조정한다.

        return out

    def backward(self, dout):
        if dout.ndim != 2:
            N, C, H, W = dout.shape
            dout = dout.reshape(N, -1)

        dx = self.__backward(dout)

        dx = dx.reshape(*self.input_shape)

        return dx

    def __backward(self, dout):
        dbeta = dout.sum(axis = 0) # 덧셈노드를  브로드캐스팅(repeat 노드를 통과)방식으로 계산하기때문에 덧셈노드의 역전파뿐 아니라 repeat노드의 역전파인 sum노드를 추가해야한다.
        dgamma = np.sum(self.xn * dout, axis = 0) # *는 adamard  product임.
        dxn = self.gamma * dout
        dxc = dxn / self.std
        dstd = -np.sum((dxn * self.xc) / (self.std * self.std), axis = 0)
        dvar = 0.5 * dstd / self.std
        dxc += (2.0 / self.batch_size) * self.xc * dvar
        dmu = np.sum(dxc, axis = 0)
        dx = dxc - dmu / self.batch_size

        self.dgamma = dgamma
        self.dbeta = dbeta

        return dx

In [ ]:
class Dropout:

    def __init__(self, dropout_ratio = 0.5):
        self.dropout_ratio = dropout_ratio
        self.mask = None

    def forward(self, x, train_flg = True): # Dropout층은 훈련시에만 적용함.
        if train_flg:
            self.mask = np.random.rand(*x.shape) > self.dropout_ratio # .rand는 0부터 1까지의 균등분포로 랜덤값을 생성함. self.mask는 true, false로 이루어진 행렬이 됨.
            return x * self.mask
        else:
            return x * (1.0 - self.dropout_ratio) # 신호량을 맞추기위해 리스케일링. 이해안감.

    def backward(self, dout):
        return dout * self.mask # 신호를 죽인부분은 미분도 죽임.

# 심층신경망

In [ ]:
class MultiLayerNet:

    def __init__(self, input_size, hidden_size_list, output_size, activation = 'relu', weight_init_std = 'relu',  # weight_init_std : 가중치의 표준편차 지정
                 weight_decay_lambda = 0, use_dropout = False, dropout_ration = 0.5, use_bachnorm = False):       # relu나 he로 지정하면 HE초기값으로, sigmoid나 xaviar로 지정하면 xaviar초기값으로 설정

        self.input_size = input_size
        self.output_size = output_size
        self.hidden_size_list = hidden_size_list
        self.hidden_layer_num = len(hidden_size_list)
        self.use_dropout = use_dropout
        self.weight_decay_lambda = weight_decay_lambda
        self.use_bachnorm = use_bachnorm
        self.params = {}

        # 가중치 초기화
        self.__init_weight(weight_init_std)

        # 계층 생성
        activation_layer = {'sigmoid' : Sigmoid, 'relu' : Relu}

        self.layers = OrderedDict()
        for idx in range(1, self.hidden_layer_num + 1):
            self.layers['Affine' + str(idx)] = Affine(self.params['W' + str(idx)], self.params['b' + str(idx)])

            if self.use_bachnorm: # batchnormalization층을 사용할 경우.
                self.params['gamma' + str(idx)] = np.ones(hidden_size_list[idx - 1]) # scail의 초기값은 1로지정.(변화없음)
                self.params['beta' + str(idx)] = np.zeros(hidden_size_list[idx - 1]) # shift의 초기값은 0으로 지정.(변화없음)
                self.layers['BatchNorm'+ str(idx)] = BatchNormalization(self.params['gamma' + str(idx)], self.params['beta' + str(idx)]) # 레이어 추가

            self.layers['Activation_function' + str(idx)] = activation_layer[activation]() # 인스턴스 생성

            if self.use_dropout:
                self.layers['Dropout' + str(idx)] = Dropout(dropout_ration)

        idx = self.hidden_layer_num + 1 # 마지막 계층은 activation층이 없기 때문에 따로 생성한다.
        self.layers['Affine' + str(idx)] = Affine(self.params['W' + str(idx)], self.params['b' + str(idx)])

        self.last_layer = SoftmaxWithLoss()

    def __init_weight(self, weight_init_std):
        """ 가중치 초기화
        weight_init_std : 가중치의 표준편차 지정
        relu나 he로 지정하면 HE초기값으로 설정, sigmoid나 xavier로 지정하면 Xavier초기값으로 지정
        """
        all_size_list = [self.input_size] + self.hidden_size_list + [self.output_size] # 자료형이 numpy array라면 덧셈연산시 벡터연산을하지만 리스트라면 단순히 옆으로 이어붙인다.
                                                                                       # mnist 예시에서는 [784, 100, 100, 100, 10]인 리스트가 생성됨
        for idx in range(1, len(all_size_list)):
            scale = weight_init_std
            if str(weight_init_std).lower() in ('relu', 'he'): # lower()은 알파벳을 소문자로 바꿔줌.
                scale = np.sqrt(2.0 / all_size_list[idx - 1])  # ReLU를 사용할때에는 HE초기값을 사용
            elif str(weight_init_std).lower() in ('sigmoid', 'xavier'):
                scale = np.sqrt(1.0 / all_size_list[idx - 1])  # sigmoid를 사용할때에는 xabier초기값을 사용

            self.params['W' + str(idx)] = scale * np.random.randn(all_size_list[idx-1], all_size_list[idx]) # 784x100행렬을 생성. 원소는 평균이 0 표준편차가 scale인 분포에서 뽑는다.
            self.params['b' + str(idx)] = np.zeros(all_size_list[idx])

    def predict(self, x, train_flg = False):
        for key, layer in self.layers.items(): # Dropout층과 BatchNorm층은 훈련시와 테스트시의 과정이 다르기 때문에  train_flg 매개변수로 구분해 준다.
            if 'Dropout' in key or 'BatchNorm' in key:
                x = layer.forward(x, train_flg)
            else:
                x = layer.forward(x)

        return x

    def loss(self, x, t, train_flg = False): # x는 입력데이터 t는 정답 레이블.

        y = self.predict(x, train_flg)

        weight_decay = 0
        for idx in range(1, self.hidden_layer_num + 2):
            W = self.params['W' + str(idx)]
            weight_decay += 0.5 * self.weight_decay_lambda * np.sum(W ** 2) # 모든 가중치의 제곱을 더한 후 lambda와 1/2를 곱하면 L2-regularization을 구할 수 있다.

        return self.last_layer.forward(y, t) + weight_decay # 크로스엔트로피와  L2-regularization을 더해서 최종적인 손실함수 값을 반환한다.

    def accuracy(self, X, T):
        Y = self.predict(X, train_flg = False)
        Y = np.argmax(Y, axis = 1)
        if T.ndim != 1 : T = np.argmax(T, axis = 1) # 라벨이 원-핫 인코딩이 돼어 있는 경우

        accuracy = np.sum(Y == T) / float(X.shape[0])

        return accuracy

    def gradient(self, x, t):

        # forward
        self.loss(x, t, train_flg = True)

        # backward
        dout = 1
        dout = self.last_layer.backward(dout)

        layers = list(self.layers.values())
        layers.reverse()
        for layer in layers:
            dout = layer.backward(dout)

        # 결과 저장
        grads = {}
        for idx in range(1, self.hidden_layer_num + 2):
            grads['W' + str(idx)] = self.layers['Affine' + str(idx)].dW + self.weight_decay_lambda * self.layers['Affine' + str(idx)].W # L2규제에 대한 미분을 더해준다(미분의 Linear한 특성을 이용.)
            grads['b' + str(idx)] = self.layers['Affine' + str(idx)].db # bias 벡터는 L2규제가 매개변수롤 weight만을 포함하고 있기때문에 L2의 미분값이 0이다.

            if self.use_bachnorm and idx != self.hidden_layer_num + 1: # 마지막층은 제외하기위해
                grads['gamma' + str(idx)] = self.layers['BatchNorm' + str(idx)].dgamma
                grads['beta' + str(idx)] = self.layers['BatchNorm' + str(idx)].dbeta

        return grads